# Imports and Reading Data

In [ ]:
import numpy as np
from random import choice
import pandas as pd
import timesfm
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score
from collections import defaultdict
import matplotlib.pyplot as plt

df = pd.read_csv(r'/content/3hour.csv', parse_dates=True)
# For topographic dataset use:
# df = pd.read_csv(r'/content/3hour_top.csv', parse_dates=True)
df.head(5)

Plot WindSpd_01HrAvg

In [ ]:
df.set_index('time')['WindSpd_01HrAvg'].plot()

# Data Preparation

## Group by station ID and extract the test set for each station

In [ ]:
# Initialize a dictionary to store the test DataFrame for each station
stations_test_splits = {}

# Group by station ID and extract the test set for each station
for station_id, station_df in df.groupby('station_id'):  # 'station_id' is the column in the DataFrame
    # Use the last 20% of each station's data as the test set
    n_total = len(station_df)
    test_df = station_df.iloc[int(0.8 * n_total):]  # The last 20% as test data

    # Store the test set for the station
    stations_test_splits[station_id] = test_df

    # Print to confirm the split
    print(f"Station {station_id}: Test set size = {len(test_df)}")

## Define Covariates and target

In [6]:
# Covariates
X_cols = [ 'Data_Time_Day__', 'Data_Time_Hour_', 'Data_Time_Month', 'Data_Time_Year_',
          'wspk10m', 'uuu30m', 'vvv10m', 'lapprs900hPa', 'uuu10m', 'wspk115m',
           'vvv850hPa', 'wspk850hPa', 'mgust', 'vvv30m', 'swdown', 'uuu500m',
           'v100m_EE', 'tdwpt_EE', 'f10m_EE', 'g10m_EE', 'gust10_inst_EE',
           'f100m_EE', 'tmax_EE', 'u100m_EE', 'u10m_EE', 'v10m_EE', 'u100m_ES',
           'f100m_ES', 'rh_ES', 'ws850p_ES', 'g10m_ES', 'u10m_ES', 'u925p_ES',
           'u850p_ES', 'f10m_ES', 'v925p_ES']
# Target Feature
Y_col =  "WindSpd_01HrAvg"

In [ ]:
for station_id, test_df in stations_test_splits.items():
    # Format the test DataFrame
    test_df.loc[:, "unique_id"] = test_df["station_id"]
    test_df.loc[:, "ds"] = test_df["time"]
    test_df.loc[:, "y"] = test_df["WindSpd_01HrAvg"]
    # Static covariates
    static_cols = ["Stn_Numeric_ID_"]

    # Dynamic covariates
    dynamic_cols = X_cols

# Defining a custom batching function which creates batches including covariates


In [8]:
def get_batched_data_fn(df, dynamic_cols, static_cols,  batch_size=64, context_len=512, horizon_len=36):
    examples = defaultdict(list)

    for station_id in df["unique_id"].unique():
        sub_df = df[df["unique_id"] == station_id]

        for start in range(0, len(sub_df) - (context_len + horizon_len), horizon_len):
            context_end = start + context_len

            # Collect inputs (target y values in the context)
            examples["unique_id"].append(station_id)
            examples["inputs"].append(sub_df["y"][start:context_end].tolist())

            # Collect dynamic covariates (exogenous variables)
            for col in dynamic_cols:
                examples[col].append(sub_df[col][start:context_end + horizon_len].tolist())

            # Collect outputs (target y values in the forecast horizon)
            examples["outputs"].append(sub_df["y"][context_end:context_end + horizon_len].tolist())

    def data_fn():
        for i in range(0, len(examples["unique_id"]), batch_size):
            yield {k: v[i:i + batch_size] for k, v in examples.items()}

    return data_fn

## Initialising TimesFM with 20 layers and 64 batch size

In [9]:
batch_size = 64
context_len = 512
horizon_len = 36

In [ ]:
tfm = timesfm.TimesFm(
    hparams=timesfm.TimesFmHparams(
        backend="cpu",
        per_core_batch_size=32,
        horizon_len=horizon_len,
        input_patch_len=32, #default and should not be changed for 200M model
        output_patch_len=128, #default and should not be changed for 200M model
        num_layers=20, #default and should not be changed for 200M model
        model_dims=1280, #default and should not be changed for 200M model
    ),
    checkpoint=timesfm.TimesFmCheckpoint(
        huggingface_repo_id="google/timesfm-1.0-200m-pytorch"
    ),
)

# Defining functions for calculating metrics

In [11]:
# Function to calculate Mean Absolute Error (MAE)
def mae(y_pred, y_true, aggregate=True):
    y_pred = np.array(y_pred)
    y_true = np.array(y_true)
    return np.mean(np.abs(y_pred - y_true))

# Function to calculate Mean Squared Error (MSE)
def mse(y_pred, y_true, aggregate=True):
    y_pred = np.array(y_pred)
    y_true = np.array(y_true)
    return np.mean(np.square(y_pred - y_true))


#Calculate R² (coefficient of determination).
def r2_score(y_true, y_pred):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    ss_res = np.sum((y_true - y_pred) ** 2)
    ss_tot = np.sum((y_true - np.mean(y_true)) ** 2)
    if ss_tot == 0:
        return float('nan')
    return 1 - (ss_res / ss_tot)

# fcast_timesfm Function
This function handles the forecasting task for a batch of time-series data using the TimesFM model. It computes forecasting metrics for each batch and aggregates the results.

In [12]:
def fcast_timesfm(input_data, dynamic_cols, xreg_mode="xreg + timesfm"):
    # Initialize aggregated metrics and forecasts
    metrics = {'mae': [], 'mse': [], 'r2': []}
    forecasts = {'cov_forecast': [], 'ols_forecast': []}

    for i, example in enumerate(input_data()):
        # Perform forecasting using TimesFM
        cov_forecast, ols_forecast = tfm.forecast_with_covariates(
            inputs=example["inputs"],
            dynamic_numerical_covariates={col: example[col] for col in dynamic_cols},
            static_categorical_covariates={"unique_id": example["unique_id"]},
            freq=[0] * len(example["inputs"]),
            xreg_mode=xreg_mode,
            ridge=0.0,
            force_on_cpu=False,
            normalize_xreg_target_per_input=True
        )

        # Convert predictions and outputs to NumPy arrays
        outputs = np.array(example["outputs"])
        cov_forecast = np.array(cov_forecast)

        # Calculate batch-level metrics
        batch_mae = mae(cov_forecast, outputs)
        batch_mse = mse(cov_forecast, outputs)
        batch_r2 = (
            r2_score(outputs, cov_forecast)
            if len(outputs) > 1 else None  # Skip R² for small batches
        )

        # Append aggregated metrics
        metrics['mae'].append(batch_mae)
        metrics['mse'].append(batch_mse)
        if batch_r2 is not None:
            metrics['r2'].append(batch_r2)

        # Store forecasts
        forecasts['cov_forecast'].append(cov_forecast)
        forecasts['ols_forecast'].append(ols_forecast)

        # Print batch metrics
        print(f"Batch {i} - MAE: {batch_mae:.2f}, MSE: {batch_mse:.2f}, R²: {batch_r2 if batch_r2 is not None else 'N/A'}")

    # Compute overall average metrics
    average_mae = np.mean(metrics['mae'])
    average_mse = np.mean(metrics['mse'])
    average_r2 = np.mean(metrics['r2']) if metrics['r2'] else None  # Handle empty R² case

    # Print overall metrics
    print(f"Overall Metrics - MAE: {average_mae:.2f}, MSE: {average_mse:.2f}, R²: {average_r2 if average_r2 is not None else 'N/A'}")

    return metrics, forecasts


# Defining Evaluation Function
This code evaluates by looping through different horizon lengths and stations, calling the fcast_timesfm function for each configuration, and compiling the results.

In [13]:
def evaluate_forecasting(
    stations_test_splits,
    dynamic_cols,
    static_cols,
    batch_size,
    context_len,
    horizon_range,
    mode
):
    """
    Evaluate forecasting performance across different horizons and stations.

    Parameters:
    - stations_test_splits: dict of station IDs to test DataFrames.
    - dynamic_cols: List of dynamic columns used in forecasting.
    - static_cols: List of static columns used in forecasting.
    - batch_size: Batch size for input data.
    - context_len: Context length for TimesFM input.
    - horizon_range: Range of horizons to evaluate (e.g., range(1, 37)).

    Returns:
    - metrics_base_df: Pandas DataFrame containing the evaluation metrics.
    """
    import numpy as np
    import pandas as pd

    metrics_list = []

    # Evaluate for each horizon length
    for horizon_len in horizon_range:
        print(f"Evaluating for Horizon Length: {horizon_len}")

        for station_id, test_df in stations_test_splits.items():
            print(f"Evaluating for Station ID: {station_id}")

            # Generate batched data with the current horizon
            input_data = get_batched_data_fn(
                test_df,
                dynamic_cols,
                static_cols,
                batch_size=batch_size,
                context_len=context_len,
                horizon_len=horizon_len
            )

            # Count batches for verification
            batch_count = sum(1 for _ in input_data())
            print(f"Horizon: {horizon_len}, Station: {station_id}, Batches: {batch_count}")

            # Perform forecasting and capture metrics
            metrics_base, forecasts = fcast_timesfm(input_data, dynamic_cols, xreg_mode=mode)

            # Calculate overall metrics for the current horizon
            mae_mean = np.mean(metrics_base['mae'])
            mse_mean = np.mean(metrics_base['mse'])
            r2_mean = np.nanmean(metrics_base['r2']) if metrics_base['r2'] else None

            # Append metrics to the results list
            metrics_list.append({
                "Horizon Length": horizon_len,
                "Station ID": station_id,
                "MAE": mae_mean,
                "MSE": mse_mean,
                "R²": r2_mean
            })

    # Convert metrics list to DataFrame
    metrics_base_df = pd.DataFrame(metrics_list)

    return metrics_base_df


# Evaluate

In [ ]:
result_xtfm_df = evaluate_forecasting(
    stations_test_splits=stations_test_splits,# Your dictionary of test splits
    dynamic_cols=dynamic_cols,                # List of dynamic columns
    static_cols=static_cols,                  # List of static columns
    batch_size=batch_size,                    # Batch size for data
    context_len=context_len,                  # Context length
    horizon_range=range(1, 6),                # Horizon range
    mode="xreg + timesfm"                     # TFM mode
)

In [ ]:
result_tfmx_df = evaluate_forecasting(
    stations_test_splits=stations_test_splits,# test splits
    dynamic_cols=dynamic_cols,                # List of dynamic columns
    static_cols=static_cols,                  # List of static columns
    batch_size=batch_size,                    # Batch size for data
    context_len=context_len,                  # Context length
    horizon_range=range(1, 6),                # Horizon range
    mode="timesfm + xreg"                     # TFM mode
)

# Scaling

Scaling the covariates

In [16]:
time_features = ['Data_Time_Day__','Data_Time_Hour_', 'Data_Time_Month', 'Data_Time_Year_']

features_to_scale = ['wspk10m', 'uuu30m', 'vvv10m', 'lapprs900hPa',
        'uuu10m', 'wspk115m', 'vvv850hPa', 'wspk850hPa', 'mgust', 'vvv30m',
        'swdown', 'uuu500m', 'v100m_EE', 'tdwpt_EE', 'f10m_EE', 'g10m_EE',
        'gust10_inst_EE', 'f100m_EE', 'tmax_EE', 'u100m_EE', 'u10m_EE',
        'v10m_EE', 'u100m_ES', 'f100m_ES', 'rh_ES', 'ws850p_ES', 'g10m_ES',
        'u10m_ES', 'u925p_ES', 'u850p_ES', 'f10m_ES', 'v925p_ES']

preprocessor = ColumnTransformer(
    transformers=[
        ('scaler', StandardScaler(), features_to_scale)
    ],
    remainder='passthrough'
)

# Fit and transform the data
scaled_data = preprocessor.fit_transform(df[features_to_scale])
scaled_df = pd.DataFrame(scaled_data, columns=features_to_scale)
scaled_df[time_features] = df[time_features]
scaled_df['y'] = df['WindSpd_01HrAvg']
scaled_df['unique_id'] = df['station_id']
scaled_df['ds'] = pd.to_datetime(df['time'])

Adding sin and cos of time components

In [17]:
def add_local_time_components(df : pd.DataFrame, utc_offset : float = 0.) -> pd.DataFrame:
    """ Adding local time and day/year components"""
    # Local time
    df = df.assign(local_time=df.index + pd.Timedelta(hours=utc_offset))
    # add year phase (cos(solstice angle) and sin(solstice))
    year_angle = (df.index.dayofyear - 172)/365.25*2*np.pi
    df = df.assign(year_cos = np.cos(year_angle),
                   year_sin = np.sin(year_angle))
    # add local time of the day (forecast time is UTC, might need to change with longitude instead)
    day_angle = (df.index.hour + utc_offset -12)/24*2*np.pi
    df = df.assign(hour_local_cos= np.cos(day_angle),
                   hour_local_sin = np.sin(day_angle))

    return df

scaled_df_timec = add_local_time_components(scaled_df.set_index('ds'), utc_offset=12.).reset_index()

In [ ]:
scaled_df_timec.head(2)

In [19]:
static_cols = ["Stn_Numeric_ID_"]
dynamic_cols = features_to_scale + time_features

In [20]:
stations_test_splits = {}
for unique_id, station_df in scaled_df_timec.groupby('unique_id'):
    test_df = station_df.iloc[int(0.8 * len(station_df)):].copy()
    stations_test_splits[unique_id] = test_df

In [21]:
# Format the test DataFrame for each station
for station_id, test_df in stations_test_splits.items():
    test_df["ds"] = scaled_df_timec.loc[test_df.index, "ds"]
    test_df["y"] = scaled_df_timec.loc[test_df.index, "y"]


# Evaluate Scaled Dataset

In [ ]:
result_base_scaled_xtfm_df = evaluate_forecasting(
    stations_test_splits=stations_test_splits,# test splits
    dynamic_cols=dynamic_cols,                # List of dynamic columns
    static_cols=static_cols,                  # List of static columns
    batch_size=batch_size,                    # Batch size for data
    context_len=context_len,                  # Context length
    horizon_range=range(1, 6),                # Horizon range
    mode="xreg + timesfm"                     # TFM mode
)

In [ ]:
result_scaled_tfmx_df = evaluate_forecasting(
    stations_test_splits=stations_test_splits,# test splits
    dynamic_cols=dynamic_cols,                # List of dynamic columns
    static_cols=static_cols,                  # List of static columns
    batch_size=batch_size,                    # Batch size for data
    context_len=context_len,                  # Context length
    horizon_range=range(1, 6),                # Horizon range
    mode="timesfm + xreg"                     # TFM mode
)

# Plotting Metrics comparing configurations

This shows no difference in metrics when scaled

In [ ]:
# Filter the DataFrames for Station ID 93831.0
metric_base_station = result_xtfm_df[result_xtfm_df["Station ID"] == 93831.0]
metric_scaled_station = result_base_scaled_xtfm_df[result_base_scaled_xtfm_df["Station ID"] == 93831.0]
metric_tfm_station = result_tfmx_df[result_tfmx_df["Station ID"] == 93831.0]
metric_scaled_tfmx_station = result_scaled_tfmx_df[result_scaled_tfmx_df["Station ID"] == 93831.0]
# Ensure the filtered DataFrames are not empty
if metric_base_station.empty or metric_scaled_station.empty or metric_tfm_station.empty:
    print("Error: Station ID 93831.0 not found in one or more DataFrames.")
else:
    # Extract the MAE All values across horizons
    base_mae_all = metric_base_station["MAE"].values  # Extract as a sequence
    scaled_mae_all = metric_scaled_station["MAE"].values  # Extract as a sequence
    tfm_mae_all = metric_tfm_station["MAE"].values  # Extract as a sequence
    scaled_tfmx_mae_all = metric_scaled_tfmx_station["MAE"].values  # Extract as a sequence
    # Plot the MAE for the specific station
    plt.figure(figsize=(10, 6))
    plt.plot(metric_base_station["Horizon Length"], base_mae_all, label="xreg + timesfm")
    plt.plot(metric_scaled_station["Horizon Length"], scaled_mae_all, label="Scaled, xreg + timesfm")
    plt.plot(metric_tfm_station["Horizon Length"], tfm_mae_all, label="timesfm + xreg")
    plt.plot(metric_scaled_tfmx_station["Horizon Length"], scaled_tfmx_mae_all, label="Scaled, timesfm + xreg")
    # Add labels, legend, and grid
    plt.xlabel("Horizon Length")
    plt.ylabel("Mean Absolute Error (MAE)")
    plt.title("MAE Comparison for Station ID: 93831.0")
    plt.legend()
    plt.grid(True)
    plt.show()

In [ ]:
result_scaled_tfmx_df['Source'] = "Scaled timesfm-xreg"
result_base_scaled_xtfm_df['Source'] = "Scaled xreg-timesfm"
result_xtfm_df['Source'] = "xreg-timesfm"
result_tfmx_df['Source'] = "timesfm-xreg"
# In case of topographic data use:
# result_scaled_tfmx_df['Source'] = "Scaled timesfm-xreg + topology"
# result_base_scaled_xtfm_df['Source'] = "Scaled xreg-timesfm + topology"
# result_xtfm_df['Source'] = "xreg-timesfm + topology"
# result_tfmx_df['Source'] = "timesfm-xreg + topology"
# Combine all DataFrames into one
combined_df = pd.concat(
    [result_scaled_tfmx_df, result_base_scaled_xtfm_df, result_xtfm_df, result_tfmx_df],
    ignore_index=True
)
# Save to CSV
combined_df.to_csv("three_hourly_metrics.csv", index=False)
print("Combined metrics saved to hour_metrics.csv")
